# Pipeline for Quality Metrics Calculation

This notebook provides an overview of the four core scripts developed to compute quality metrics using BAM/CRAM files:
- samtools_stats.py   → alignment statistics
- depth.py            → per-position depth
- plot.py             → visual coverage plot
- coverage_metrics.py → coverage metrics

The scripts form a sequential pipeline where each step depends on the previous one, ultimately producing both a statistics and coverage metrics report, as well as a graphical representation of the sequencing depth.

The pipeline requires:
- A BAM/CRAM file (alignment data)
- A BED file (regions of interest)

In the following section, it will demonstrate how to use the provided scripts to process genomic data. As part of this demonstration, it will first need to download an example CRAM file from the 1000 Genomes Project. To do so, please run the command below in your terminal to retrieve the file directly from the public AWS S3 bucket without authentication:

In [ ]:
!aws s3 cp s3://1000genomes/phase3/data/HG00099/exome_alignment/HG00099.mapped.ILLUMINA.bwa.GBR.exome.20130415.bam.cram . --no-sign-request

Another required file for this workflow is a BED file containing the genomic coordinates of the gene of interest. In the following code block, a BED file is automatically generated by retrieving gene information from the Ensembl database. This file will then be used in the subsequent steps of the demonstration.

In [7]:
import requests

# Gene Name
gene_name = "BRCA1"

# URL for ensembl
url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene_name}?content-type=application/json"

# API request
response = requests.get(url)

# Verificar se correu bem
if not response.ok:
    print(f"Erro with data for {gene_name}.")
else:
    data = response.json()
    chrom = data["seq_region_name"]
    start = data["start"]
    end = data["end"]
    name = data["display_name"]
    exon = 'Exon'
    size = data['end'] - data['start']

    # BED File Creation
    with open("genes.bed", "a") as f:
        f.write(f"{chrom}\t{start}\t{end}\t{name}\t{exon}\t{size}\n")

After securing both required files, it is now possible to proceed with the pipeline. The general workflow can be represented as:

```arduino
BAM/CRAM + BED ──▶ samtools_stats.py ──▶ depth.py ──▶ plot.py ──▶ coverage_metrics.py ──▶ metrics report + coverage plot
```

Therefore, these scripts are interconnected in a very peculiar way. Firstly, the statistics in the BAM/CRAM file are collected, allowing the quality of these data to be analyzed. Then, the BAM/CRAM file is filtered by a BED file and, therefore, the calculation of depth and, consequently, the coverage metrics are only carried out for the interval present in this file. In addition, an interactive coverage plot is generated.

**samtools_stats.py**

To extract a comprehensive statistical summary of a BAM/CRAM file, one of the backend modules relies on the `pysam` library, specifically the **pysam.samtools.stats()** function, which provides a Python interface to the `samtools stats` command-line utility, offering improved speed and less computational demand. This allows for automated, reproducible extraction and post-processing of key sequencing metrics.

It provides key indicators such as:
- Total Reads
- Total Bases
- Average Base Quality
- Read Length (Max)
- Reads Mapped (%)
- Mapping Quality 0 (%)
- Error rate
- Insert Size (Avg)
- Insert Size (Std)
- Reads paired (%)
- Reads properly paired (%)
- Duplicate Reads (%)
- Proportion of duplicated reads

The script calls `pysam.samtools.stats()` to execute the underlying command, capturing its multi-line text output for parsing. The extracted data includes both directly reported values, such as total reads, reads mapped, and average base quality, as well as values that need to be calculated, namely mapping quality and percentage of duplicated reads. After computation, the metrics are transformed into a pandas DataFrame.

**depth.py**

Depth of coverage and breadth of coverage are critical metrics for assessing the reliability of sequencing data in specific genomic regions. This module extracts the depth information for each base indicated in the regions defined in a BED file, enabling a detailed evaluation of coverage uniformity across gene or exons of interest.

This script starts to process the information in the BED file in order to create a pandad dataframe with key genomic coordinates, such as chromosome, start position, end position, and gene/exon labels if gene or exon were selected.

After creating the BED format dataframe, it is possible to extract the depth. The depth extraction is implemented by three different methods: `samtools depth`, a command line tool that computes the read depth; `pysam.depth`, a Python module that replicates the functionality of samtools depth; and `pysam.coverage`, a Python-based method that provides a fast and efficient way to calculate sequencing depth.

In the end, a pandas DataFrame is created with chromosome, position and the respective depth in every depth extraction method. The result can be stored as a tabular file.

**plot.py**

The aim of this script is to create a coverage plot that enables viewing the distribution of coverage for a specific genomic region. Thus, is responsible for generating an interactive depth of coverage visualization. In order to establish a proper workflow, this function relates to the script described above. That way, after having the depth information, the resulting values are annotated with gene and exon identifiers provided by the BED input.

The core of the plotting logic builds three main traces: the coverage line, the mean coverage line, and a line representing the quality threshold. To enhance interpretability, the function display discontinuities between non-contiguous positions, avoiding misleading connections across genomic gaps. In addition, regions where coverage falls below the threshold are highlighted by a shaded red area. Exon regions are emphasized by overlaying transparent rectangles defined in the shapes attribute of the Plotly layout. In the end, the plot can be returned for display or saved to disk.

**coverage_metrics.py**

After computing depth extraction, it is essential to translate this information into interpretable coverage metrics. That way, this module calculates metrics such as average, minimum, maximum and median read depth, depth of coverage at different thresholds (1x, 10x, 15x, 20x, 30x, 50x, 100x, 500x), and breadth of coverage. These metrics are fundamental for assessing the reliability of variant detection and overall sequencing quality. Moreover, this module aggregates the summary statistics computed earlier to have a full and unique report for each BAM/CRAM file.

Like in the previous modules, all these metrics are stored in a pandas DataFrame. This way, it ensures that all the metrics calculated are easily compared, aggregated, or exported for downstream quality control and reporting.

Contrarily to the samtools stats module, this script extends the coverage assessment to a different level by implementing functions that calculate metrics for individual genes and exons.

Finally, as reported, this script functions as an integrative module that connects all the scripts described above. This way, this script imports the quality metrics extracted by *samtools_stats*, extracts depth information, generates the interactive plot and then applies the coverage metrics calculation.

In [ ]:
from backend.coverage_metrics import calculate_metrics
from backend.depth import filter_bed

# Path for required files
cram_path = 'HG00099.mapped.ILLUMINA.bwa.GBR.exome.20130415.bam.cram'
bed_path = 'genes.bed'
ref_cache = './data/reference_genome'


# Step 1: Create BED_df
bed_df = filter_bed(bed_path)
print(bed_df)

metrics_df = calculate_metrics(cram_path, bed_df, stats=True, ref_cache=ref_cache)
print(metrics_df)

However, the scripts provided can be used in two ways: either as standalone modules, which can be imported and called individually, or integrated directly into the full pipeline. This flexibility allows users to test specific functions or run the complete workflow depending on their needs:

In [ ]:
# Import backend modules
from backend.samtools_stats import calculate_stats
from backend.depth import filter_bed, get_depth_data
from backend.plot import coverage_plot
from backend.coverage_metrics import calculate_coverage_metrics
import plotly.graph_objects as go

# Path for required files
cram_path = 'HG00099.mapped.ILLUMINA.bwa.GBR.exome.20130415.bam.cram'
bed_path = 'genes.bed'
ref_cache = './data/reference_genome'

# Statistical metrics extraction
stats_df = calculate_stats(cram_path, ref_cache)
print(stats_df)

# Depth extraction
bed_df = filter_bed(bed_path, gene_selection='BRCA1')
depth_df = get_depth_data(cram_path, bed_df)

# Coverage plot
fig = coverage_plot(cram_path, bed_df)
fig.show()

# Coverage metrics calculation
coverage_df = calculate_coverage_metrics(depth_df['DEPTH'], bed_df['SIZE'].sum(), thresholds = [1, 10, 15, 20, 30, 50, 100, 500])
print(coverage_df)

This approach allows the same functions to be used directly on command line or implemented in an application, while maintaining clear traceability and reproducibility of the bioinformatics analyses.

**Example of usage**:

```bash
python coverage_metrics.py <cram_path> <bed_path> <output_path> --gene_selection <GENE_NAME> --exon_selection <EXON_NUMBER> --method <method> --stats --output_plot <output_plot_path>
```